In [2]:
from io_func import *

In [6]:
unit = {
    'Provinsi - Industri':'provinsi_industri',
    'Provinsi - Lapangan Usaha':'provinsi_lapus',
    'Pulau - Industri':'pulau_industri',
    'Pulau - Lapangan Usaha':'pulau_lapus',
    'Industri':'industri',
    'Pulau':'pulau',
    'Provinsi':'provinsi',
    'Lapangan Usaha':'lapus'
}
unit_opt = [{'label':key, 'value':item} for key,item in unit.items()]

In [9]:
x = [1,2,3]
x[1:]

[2, 3]

In [22]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Selamat Datang di VIRIO_2016'),
    html.Hr(),
    html.H3('Pilih Unit Penyedia dan Unit Pengguna'),
    html.Div([
        html.Div([
            html.Label([
                "Penyedia",
                dcc.Dropdown(
                    id="supply-unit", clearable = False,
                    options = supply_user_opt, value = supply_user_opt[0]['value']
                )
            ])
        ], style = {'display':'inline-block', 'width':'49%'}),
        html.Div([
            html.Label([
                "Pengguna",
                dcc.Dropdown(
                    id="use-unit", clearable = False,
                    options = supply_user_opt, value = supply_user_opt[0]['value']
                )
            ])
        ], style = {'display':'inline-block', 'width':'49%'})
    ]),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id = 'agg-type',
            options = [
                {'label':'Proporsi output yang digunakan', 'value':'output'},
                {'label':'Proporsi penggunaan input', 'value':'input'},
            ],
            value = 'output',
            labelStyle = {'display':'block', 'margin-left':'auto', 'margin-right':'auto'}
        )
    ]),
    html.Div([
        dcc.Dropdown(
            id = "unit-name", clearable = False
        )
    ]),
    html.Hr(),
    html.Hr(),
    html.H1(id="heading-1"),
    html.H2(id="heading-2"),
    dcc.Graph(id="io-used-graph")
])

@app.callback(
    Output('unit-name', 'options'),
    Output('unit-name', 'value'),
    Input('supply-unit', 'value'),
    Input('use-unit', 'value'),
    Input('agg-type', 'value')
)

def update_dropdown(supply_unit, use_unit, agg_type):
    unit = use_unit if agg_type == 'input' else supply_unit

    if len(unit.split('_')) > 1:
        unit_list = unit.split('_')
        labels = [
            f'{unit1} - {unit2}' for unit1, unit2 in zip(
                tables[f"kode_{unit_list[0]}"]["data"][tables[f"kode_{unit_list[0]}"]["attr"]["nama"]].to_list(),
                tables[f"kode_{unit_list[1]}"]["data"][tables[f"kode_{unit_list[1]}"]["attr"]["nama"]].to_list()
            )
        ]
        values = [
            f'{unit1};{unit2}' for unit1, unit2 in zip(
                tables[f"kode_{unit_list[0]}"]["data"][tables[f"kode_{unit_list[0]}"]["attr"]["kode"]].to_list(),
                tables[f"kode_{unit_list[1]}"]["data"][tables[f"kode_{unit_list[1]}"]["attr"]["kode"]].to_list()
            )
        ]
        
    else :
        labels = tables[f"kode_{unit}"]["data"][tables[f"kode_{unit}"]["attr"]["nama"]].to_list()
        values = tables[f"kode_{unit}"]["data"][tables[f"kode_{unit}"]["attr"]["kode"]].to_list()
        
    values = [f'{unit};{value}' for value in values]
    options = [{'label':label, 'value':value} for label, value in zip(labels, values)]
    return options, options[0]['value']

@app.callback(
    Output('heading-1', 'children'),
    Output('heading-2', 'children'),
    Input('supply-unit', 'value'),
    Input('use-unit', 'value'),
    Input('agg-type', 'value'),
    Input('unit-name', 'value')
)
def update_graph(supply_unit, use_unit, agg_type, unit_name):
    units = unit_name.split(';')[0].split('_')
    unit_values = unit_name.split(';')[1:]
    condition = {}
    status = 'penyedia' if agg_type == 'output' else 'pengguna'
    
    for unit, unit_value in zip( units, unit_values ):
        condition[f'{unit}_{status}'] = unit_value
    
    conn = get_connection()
    try :
        print(pd.read_sql(io_used_per_unit(supply_unit, use_unit, agg_type, condition), conn, params = condition))
    except Exception as e:
        conn.close()
        print(e)
    return 'csd', 'cws'

app.run_server(mode = 'inline', debug = True, port = '8051')

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_11984/140571358.py in update_graph(
    supply_unit='provinsi_industri',
    use_unit='provinsi_industri',
    agg_type='output',
    unit_name=None
)
     92 )
     93 def update_graph(supply_unit, use_unit, agg_type, unit_name):
---> 94     units = unit_name.split(';')[0].split('_')
        units = undefined
        unit_name.split.split = undefined
     95     unit_values = unit_name.split(';')[1:]
     96     condition = {}

AttributeError: 'NoneType' object has no attribute 'split'

     provinsi_penyedia industri_penyedia provinsi_pengguna industri_pengguna  \
0                   11              I-01                11              I-13   
1                   11              I-01                12              I-13   
2                   11              I-01                11          